# Wikepedia arabic articles scrapping

In this project I intend to create a scraping tool for wikepedia where the scrapper get the paragraphs in a certain article as well as collecting all the hyperlinks for other articles inside the body and store them in a Dataframe. In a future version I will add the ability to scrap the links I collect to create a dataset of many articles relevant to the first article I started with.

Please use this code responsibly and review wikepedia's rules for scrapping to check the legality and the effect of this method before usage.

## Imports

In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

## Access the article using URL

In [2]:
url = "https://ar.wikipedia.org/wiki/%D8%A7%D9%84%D9%86%D8%A7%D8%AF%D9%8A_%D8%A7%D9%84%D8%A3%D9%87%D9%84%D9%8A_(%D9%85%D8%B5%D8%B1)"
soup = BeautifulSoup(requests.get(url).text, 'html.parser')

#print(soup.prettify()[:15000]) # To visualise the soup object uncomment this line.

## Extract information from BeautifulSoup object

### Find title of article

In [3]:
title = soup.find('h1', {'id' : 'firstHeading'})
print('article title: {}.'.format(title.get_text()))

article title: النادي الأهلي (مصر).


### Filter page body and get text

In [4]:
# We will get the body as I am only intrested in the information inside the body
body = soup.find(class_ = 'mw-content-rtl mw-parser-output')

In [5]:
# Filter the body to only keep paragraphs in the main text of the article
infobox = body.find(class_ = 'infobox')
infobox_v2 = body.find(class_ = 'infobox infobox_v2')
navigation = body.find(class_ = 'navbox')
hatnote = body.find(class_ = 'hatnote navigation-not-searchable')
tables = body.find(class_ = 'wikitable')

infobox.decompose() # Remove side info box
infobox_v2.decompose() # Remove side info box
navigation.decompose() # Remove navigation elements
hatnote.decompose() # Remove hat note
tables.decompose() # Remove tables


In [6]:
# Get all text in the article body 
text_body = body.get_text()

## Get articles listed categories

In [7]:
cat_links = soup.find(id = 'mw-normal-catlinks').find_all('li') # Get div the items inside the categotiries list

categories_list = [category_element.get_text() for category_element in cat_links ] # Iterate over all the elements in the categories list and get the element's title text

In [8]:
print(categories_list)

['النادي الأهلي (مصر)', 'أندية الدوري المصري الممتاز لكرة القدم', 'أندية رياضية في مصر', 'أندية كرة قدم أسست في 1907', 'أندية كرة قدم غير هابطة', 'أندية كرة قدم في القاهرة', 'أندية كرة قدم في مصر', 'أندية متعددة الرياضات', 'تأسيسات سنة 1907 في مصر', 'فرق رياضية في مصر', 'مباريات النادي الأهلي (مصر)', 'نواد وجمعيات في مصر']


### Get links from page to scrape them

In [9]:
a_tags = body.find_all('a')

#### Filter links
Now we go throug the href links to filter out images and construct the full URLs.

In [10]:
print(a_tags[10])

<a href="/wiki/%D8%A7%D9%84%D9%86%D8%A7%D8%AF%D9%8A_%D8%A7%D9%84%D8%A3%D9%87%D9%84%D9%8A_%D8%A7%D9%84%D9%85%D8%B5%D8%B1%D9%8A_%D9%84%D9%83%D8%B1%D8%A9_%D8%A7%D9%84%D9%85%D8%A7%D8%A1" title="النادي الأهلي المصري لكرة الماء"><img class="mw-file-element" data-file-height="300" data-file-width="300" decoding="async" height="50" src="//upload.wikimedia.org/wikipedia/commons/thumb/0/0e/Water_polo_pictogram.svg/50px-Water_polo_pictogram.svg.png" srcset="//upload.wikimedia.org/wikipedia/commons/thumb/0/0e/Water_polo_pictogram.svg/75px-Water_polo_pictogram.svg.png 1.5x, //upload.wikimedia.org/wikipedia/commons/thumb/0/0e/Water_polo_pictogram.svg/100px-Water_polo_pictogram.svg.png 2x" width="50"/></a>


In [11]:
articles_path = list() # articles path store the path to other wikepedia articles that the current one contains

# Be advised that the condition in data_constructor.py is updated and more accurate.
for path in a_tags:
    try:
        if path.get('title') == None or 'توضيح' in path.get('title'):
            continue
        if path.get('href') != None and path.get('class') == None and 'cite_note' not in path.get('href'):
            article_name = path.get('title')
            full_url = 'https://ar.wikipedia.org' + path.get('href')
            value = (article_name , full_url)
            if value not in articles_path: # Check if this article exists in the list of links
                articles_path.append(value)
    except KeyError:
        continue

print(len(articles_path))

1033


In [12]:
articles_linkes_df = pd.DataFrame.from_records(articles_path, columns= ['name', 'URL'])
articles_linkes_df.head()

,name,URL
0,النادي الأهلي (مصر),https://ar.wikipedia.org/wiki/%D8%A7%D9%84%D9%...
1,النادي الأهلي للكرة الطائرة (مصر),https://ar.wikipedia.org/wiki/%D8%A7%D9%84%D9%...
2,النادي الأهلي للكرة الطائرة للسيدات,https://ar.wikipedia.org/wiki/%D8%A7%D9%84%D9%...
3,النادي الأهلي المصري لكرة السلة,https://ar.wikipedia.org/wiki/%D8%A7%D9%84%D9%...
4,النادي الأهلي لكرة السلة للسيدات,https://ar.wikipedia.org/wiki/%D8%A7%D9%84%D9%...


## Exmaple for using the article scraper python script

In [13]:
from arabic_wikipedia_scraper import WikipediaArabicArticleScraper

scraper = WikipediaArabicArticleScraper()
article_data = scraper.scrape_article(url, save_as_json = True) # Returns a dict.

Soup object created!
json file saved!


In [14]:
print(article_data)

{'title': 'النادي الأهلي (مصر)', 'url': 'https://ar.wikipedia.org/wiki/%D8%A7%D9%84%D9%86%D8%A7%D8%AF%D9%8A_%D8%A7%D9%84%D8%A3%D9%87%D9%84%D9%8A_(%D9%85%D8%B5%D8%B1)', 'body': 'الفرق الرياضية بالنادي الأهلي\n\n\nكرة قدم\n\nالكرة الطائرة (رجال)\n\nالكرة الطائرة (سيدات)\n\n\nكرة السلة (رجال)\n\nكرة السلة (سيدات)\n\nكرة الماء\n\n\nكرة اليد\n\nالهوكى\n\nهوكى الانزلاق\n\nالنَّادِي الأَهْلِي لِلْرِّيَاضَةِ البَدَنِيَة أو كَما يُعرف اختصارًا بِاسم النَّادِي الأَهْلِي، هو نادٍ رياضي مصري محترف يلعب في الدوري المصري الممتاز، ومقره في القاهرة، وهو النادي الوحيد في مصر بجانب نادي الزمالك الذي لم يهبط إلى دوري الدرجة الثانية.[13][14][15] تأسَّس النادي الأهلي بتاريخ 24 أبريل 1907[1][2][16] وبعد ذلك أنشئ مجلس إدارة النادي الأهلي برئاسة الإنجليزي ميشيل إِنِس،[17] وفي 1909 أُسِّس أول ملعب كرة قدم في تاريخ النادي وسُمِّيَ حينها باسم (الحُوش)، وتطور هذا الملعب حتَّى أصبح بمُسمَّاه الحالي (ملعب مختار التتش)،[18] وأُنشئ أول فريق كرة قدم عام 1911، ووصل عدد الألعاب داخل النادي الأهلي عام 1916 إلى 4 ألعاب هي

## Example for using the Dataset constructor python script

In [15]:
from dataset_constructor import DatasetConstructor

dataset_maker = DatasetConstructor()
dataset_maker.create_dataset(url)

Soup object created!
Number of a tags: 6503
Numbers of urls in the output dataframe:  1043
